# Resample one month

After rename, then each file is resampled.

In [5]:
import os
import pandas as pd
import datetime as datetime

In [6]:
def resample(df, sampling_time, file_name):
    header_names = ['time (s)', 
                    'Ia (A)',
                    'Ib (A)',
                    'Ic (A)',
                    'In (A)',
                    'Iavg (A)',
                    'Va (V)',
                    'Vb (V)',
                    'Vc (V)',
                    'Vavg (V)',
                    'Pa (kW)',
                    'Pb (kW)',
                    'Pc (kW)',
                    'Ptot (kW)',
                    'VARa (kVAR)',
                    'VARb (kVAR)',
                    'VARc (kVAR)',
                    'VARtot (kVAR)',
                    'VAa (kVA)',
                    'VAb (kVA)',
                    'VAc (kVA)',
                    'VAtot (kVA)',
                    'PFa',
                    'PFb',
                    'PFc',
                    'PFtot',
                    'DFa',
                    'DFb',
                    'DFc',
                    'DFtot']
    
    # Step 1: offset time to start at 0
    min_time = min(df['time (s)'])
    df['time (s)'] = df['time (s)'] - min_time

    # Step 2: compute time difference
    df['time diff (s)'] = df['time (s)'].diff(periods=1)
    df.loc[0, 'time diff (s)'] = 0
    df['time diff period'] = 0

    # Step 3: separate into dataframe
    df1 = df[df['time (s)'] < sampling_time*60/2]
    df2 = df[(df['time (s)'] >= sampling_time*60/2) & (df['time (s)'] < 86400 - sampling_time*60/2)]

    df2 = df2.copy()
    df2['time (s)'] = df2['time (s)'] + sampling_time*60/2
    df2['time diff period'] = df2['time (s)'] // (sampling_time*60)
    df2['time (s)'] = df2['time (s)'] - sampling_time*60/2

    # Step 4: concat df1 and df2
    df = pd.concat([df1, df2])

    df3 = df.copy()
    
    # Step 5: multiply each column by time difference
    for col in header_names[1:]:
        df3[col] = df3[col]*df3['time diff (s)']

    # Step 6: sum the value in the same time period
    df3 = df3.groupby(df['time diff period']).sum(header_names[1:])

    # Step 7: divide each value by total time
    for col in header_names[1:]:
        df3[col] = df3[col]/df3['time diff (s)']
        
    # Step 8: set time index
    df3 = df3.set_index('time (s)')
    df3 = df3.reset_index() # reset index
    df3['time (s)'] = df3.index * sampling_time # sampling time every 15 minutes

    # Step 9: Prepare to create datetime columns
    numeric_part = file_name[-12:-4]

    # separate the date into year, month, and day
    year = int(numeric_part[:4])
    month = int(numeric_part[4:6])
    day = int(numeric_part[6:8])
    date = pd.Timestamp(year, month, day)

    # Step 10: add date time column
    df3['time (s)'] = date + pd.to_timedelta(df3['time (s)'], unit='m')

    # Step 11: change column name form time (s) to datetime
    df3 = df3.rename(columns={'time (s)': 'datetime'})

    # Step 12: drop time diff column
    df3 = df3.drop('time diff (s)', axis=1)
    
    df3 = df3.drop('time diff period', axis=1)
    
    return df3

In [7]:
def no_resample(df, file_name):
    
    # Step 1: offset time to start at 0
    min_time = min(df['time (s)'])
    df['time (s)'] = df['time (s)'] - min_time
    # Step 2: compute time difference
    df['time diff (s)'] = df['time (s)'].diff(periods=1)
    df.loc[0, 'time diff (s)'] = 0
    df['time diff period'] = 0

    # Step 9: Prepare to create datetime columns
    numeric_part = file_name[-12:-4]
    # separate the date into year, month, and day
    year = int(numeric_part[:4])
    month = int(numeric_part[4:6])
    day = int(numeric_part[6:8])
    date = pd.Timestamp(year, month, day)
    # Step 10: add date time column
    df['time (s)'] = date + pd.to_timedelta(df['time (s)'], unit='s')

    # Step 11: change column name form time (s) to datetime
    df = df.rename(columns={'time (s)': 'datetime'})

    # Step 12: drop time diff column
    df = df.drop('time diff (s)', axis=1)
    
    df = df.drop('time diff period', axis=1)
    
    return df

In [8]:
def patch_dataframe(df, start_datetime, end_datetime, sampling_time):
    header_names = ['time (s)', 
                    'Ia (A)',
                    'Ib (A)',
                    'Ic (A)',
                    'In (A)',
                    'Iavg (A)',
                    'Va (V)',
                    'Vb (V)',
                    'Vc (V)',
                    'Vavg (V)',
                    'Pa (kW)',
                    'Pb (kW)',
                    'Pc (kW)',
                    'Ptot (kW)',
                    'VARa (kVAR)',
                    'VARb (kVAR)',
                    'VARc (kVAR)',
                    'VARtot (kVAR)',
                    'VAa (kVA)',
                    'VAb (kVA)',
                    'VAc (kVA)',
                    'VAtot (kVA)',
                    'PFa',
                    'PFb',
                    'PFc',
                    'PFtot',
                    'DFa',
                    'DFb',
                    'DFc',
                    'DFtot']
    
    extended_time = pd.date_range(start=start_datetime, end=end_datetime, freq=str(sampling_time)+'T')
    
    extended_df = pd.DataFrame({'datetime': extended_time})

    # Perform linear interpolation
    extended_df = extended_df.merge(df, on='datetime', how='left')
    
    for i in range(1, len(header_names)):
        extended_df[header_names[i]] = extended_df[header_names[i]].interpolate(method='linear')
        
    return extended_df

In [9]:
month_list = ['January2024','February2024']

# Create lists to store the file names, number of index in the original data, 
# number of index in the 5-minute resampled data, and number of index in the 15-minute resampled data
# to be used for determining the missing data
file_name_list = []
num_index_original = []
num_index_5min = []
num_index_15min = []

for month in month_list:
    print(month)
    folder_path = f"/Users/nattapongkongkaew/Desktop/input_data/rename_filename/data_load/{month}/"  # Replace this with the actual path to your folder

    except_list = []
    
    file_names = os.listdir(folder_path)
    sorted_file_names = sorted(file_names)
    
    for file_name in sorted_file_names:
        if file_name.startswith("load") and file_name.endswith(".csv"):
            
            try:
                header_names = ['time (s)', 
                    'Ia (A)',
                    'Ib (A)',
                    'Ic (A)',
                    'In (A)',
                    'Iavg (A)',
                    'Va (V)',
                    'Vb (V)',
                    'Vc (V)',
                    'Vavg (V)',
                    'Pa (kW)',
                    'Pb (kW)',
                    'Pc (kW)',
                    'Ptot (kW)',
                    'VARa (kVAR)',
                    'VARb (kVAR)',
                    'VARc (kVAR)',
                    'VARtot (kVAR)',
                    'VAa (kVA)',
                    'VAb (kVA)',
                    'VAc (kVA)',
                    'VAtot (kVA)',
                    'PFa',
                    'PFb',
                    'PFc',
                    'PFtot',
                    'DFa',
                    'DFb',
                    'DFc',
                    'DFtot']

                csv_file_path = folder_path + file_name
                # csv_file_path = f'/Users/nattapongkongkaew/Desktop/input_data/actual_data_received_from_meter_copy/load_data/{month}/{file_name}'
                
                if not os.path.isfile(csv_file_path):
                    print(f"File '{csv_file_path}' not found.")
                    
                else:
                    # read csv file as pandas dataframe
                    df = pd.read_csv(csv_file_path)
                    
                    if len(df.columns.values) == len(header_names):
                        df = pd.read_csv(csv_file_path, header=None)
                        df.columns = header_names 

                    else:
                        df = pd.read_csv(csv_file_path, skiprows=1, header= None)
                        df.columns = header_names
                        
                print(file_name)
                
                df_original = df.copy()  
                
                # Error handling for resetting meter
                max_time = max(df_original['time (s)'])
                max_index = df_original[df_original['time (s)'] == max_time].index[0]

                if max_index != len(df_original) - 1:
                    df_original1 = df_original[df_original.index <= max_index]
                    df_original2 = df_original[df_original.index > max_index]
                    df_original2['time (s)'] += max_time
                    
                    df_original = pd.concat([df_original1, df_original2])
                
                df_5min = df_original.copy()
                df_15min = df_original.copy()
                
                # resample the data for 5-minute and 15-minute intervals
                df_5min = resample(df_5min, 5, file_name)
                df_15min = resample(df_15min, 15, file_name)
                
                # keep the original data without resampling
                df_original = no_resample(df_original, file_name)
                
                # Path to save the resampled data. 
                # Note that: In each directory must have subdirectories for each month.
                folder_path_original = f"/Users/nattapongkongkaew/Desktop/input_data/preprocessed_data/data_load_noresample/{month}/"
                folder_path_5min = f"/Users/nattapongkongkaew/Desktop/input_data/preprocessed_data/data_load_5minresample/{month}/"
                folder_path_15min = f"/Users/nattapongkongkaew/Desktop/input_data/preprocessed_data/data_load_15minresample/{month}/"
                
                file_name_list.append(file_name)
                num_index_original.append(len(df_original))
                num_index_5min.append(len(df_5min))
                num_index_15min.append(len(df_15min))
                
                # Error Handling for missing data during the night
                if len(df_5min) == 288:
                    df_original.to_csv(f'{folder_path_original}{file_name}', index=False)
                    df_5min.to_csv(f'{folder_path_5min}{file_name}', index=False)
                    df_15min.to_csv(f'{folder_path_15min}{file_name}', index=False)
                
                # It is better to investigate the file one-by-one to determine the missing data. So, I commented the code below instead.
                    
                # else: # patch the missing data with linear interpolation
                #     datetime_list_5min = list(df_5min['datetime'])
                #     datetime_list_15min = list(df_15min['datetime'])
                    
                #     start_datetime_5min = str(datetime_list_5min[0])
                #     [start_date, start_time] = start_datetime_5min.split(' ')
                #     end_time_5min = '23:55:00'
                #     end_datetime_5min = start_date + ' ' + end_time_5min
                    
                #     start_datetime_15min = str(datetime_list_15min[0])
                #     [start_date, start_time] = start_datetime_15min.split(' ')
                #     end_time_15min = '23:45:00'
                #     end_datetime_15min = start_date + ' ' + end_time_15min
                    
                #     df_5min = patch_dataframe(df_5min, start_datetime_5min, end_datetime_5min, 5)
                #     df_15min = patch_dataframe(df_15min, start_datetime_15min, end_datetime_15min, 15)
                    
                #     df_original.to_csv(f'{folder_path_original}{file_name}', index=False)
                #     df_5min.to_csv(f'{folder_path_5min}{file_name}', index=False)
                #     df_15min.to_csv(f'{folder_path_15min}{file_name}', index=False)
                    
                #     file_name_list.append(file_name)
                #     num_index_original.append(len(df_original))
                #     num_index_5min.append(len(df_5min))
                #     num_index_15min.append(len(df_15min))
            
            except Exception as e:
                except_list.append((file_name,e)) 
                
    print(except_list)
    print('----------------------------------')

January2024
load_gwz_20240101.csv
load_gwz_20240102.csv
load_gwz_20240103.csv
load_gwz_20240104.csv
load_gwz_20240105.csv
load_gwz_20240106.csv
load_gwz_20240107.csv
load_gwz_20240108.csv
load_gwz_20240109.csv
load_gwz_20240112.csv
load_gwz_20240113.csv
load_gwz_20240114.csv
load_gwz_20240115.csv
load_gwz_20240116.csv
load_gwz_20240117.csv
load_gwz_20240118.csv
load_gwz_20240119.csv
load_gwz_20240120.csv
load_gwz_20240121.csv
load_gwz_20240122.csv
load_gwz_20240123.csv
load_gwz_20240124.csv
load_gwz_20240125.csv
load_gwz_20240126.csv
load_gwz_20240127.csv
load_gwz_20240128.csv
load_gwz_20240129.csv
load_gwz_20240130.csv
load_gwz_20240131.csv
[]
----------------------------------
February2024
load_gwz_20240201.csv
load_gwz_20240202.csv
load_gwz_20240203.csv
load_gwz_20240204.csv
load_gwz_20240205.csv
load_gwz_20240206.csv
load_gwz_20240207.csv
load_gwz_20240208.csv
load_gwz_20240210.csv
load_gwz_20240211.csv
load_gwz_20240212.csv
load_gwz_20240213.csv
load_gwz_20240215.csv
load_gwz_2024

# Check if there is missing data or not

In [10]:
check_data = {'name': file_name_list, 
        'num_index_original': num_index_original, 
        'num_index_5min': num_index_5min,
        'num_index_15min': num_index_15min}
df_check_data = pd.DataFrame(check_data)
df_check_data

,name,num_index_original,num_index_5min,num_index_15min
0,load_gwz_20240101.csv,213413,288,96
1,load_gwz_20240102.csv,213023,288,96
2,load_gwz_20240103.csv,212124,288,96
3,load_gwz_20240104.csv,212124,288,96
4,load_gwz_20240105.csv,212766,288,96
5,load_gwz_20240106.csv,213324,288,96
6,load_gwz_20240107.csv,213325,288,96
7,load_gwz_20240108.csv,213323,288,96
8,load_gwz_20240109.csv,213036,288,96
9,load_gwz_20240112.csv,213238,288,96


## The missing data causes the number of index in (original < 120000) or (5min_resample != 288)

In [11]:
# filter the row that has num_index_5min < 288
df_check_data = df_check_data[(df_check_data['num_index_original'] < 120000) | (df_check_data['num_index_5min'] != 288)]

# sort the dataframe by num_index_5min
df_check_data = df_check_data.sort_values(by=['name'])
df_check_data

,name,num_index_original,num_index_5min,num_index_15min
11,load_gwz_20240114.csv,213446,287,96
25,load_gwz_20240128.csv,211526,287,96
38,load_gwz_20240211.csv,212179,287,96
52,load_gwz_20240226.csv,211977,287,96


# The missing data occurs in these following files.

In [12]:
df_check_data['name']

11    load_gwz_20240114.csv
25    load_gwz_20240128.csv
38    load_gwz_20240211.csv
52    load_gwz_20240226.csv
Name: name, dtype: object